2025.05.12, Zhang Chao

In [1]:
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import os
import sys
import scipy
sys.path.append('/home/climate/chaoz/code/utils/')
from plot_utils import plot_settings


In [2]:
os.chdir('/home/climate/chaoz/project/03Irr_Ts_CN/processed/')
dLST_win21 = xr.open_dataset('delta_LSTday_Yr_CN_2001_2020_win21.nc').sel(time=slice('2010','2010'))
dLST_win31 = xr.open_dataset('delta_LSTday_Yr_CN_2001_2020.nc').sel(time=slice('2010','2010'))
dLST_win41 = xr.open_dataset('delta_LSTday_Yr_CN_2001_2020_win41.nc').sel(time=slice('2010','2010'))


In [3]:
ds = xr.merge([dLST_win21.squeeze('time').rename({'Ts': 'T21'}).T21,
               dLST_win31.squeeze('time').rename({'Ts': 'T31'}).T31,
               dLST_win41.squeeze('time').rename({'Ts': 'T41'}).T41])

df = ds.to_dataframe().reset_index().dropna().drop(columns=['time','lat','lon'])

In [4]:
def getSigFlag(p):
    """Get the significance sign (star)

    Args:
        p (float32): p-value

    Returns:
        str: significance sign
    """
    strSig = ''
    if p<0.05:
        strSig = '**'
    elif p<0.1:
        strSig = '*'
    else:
        strSig = ''
    return strSig


def plot_scatter(ax,x,y,xlabel,ylabel):
    ax.scatter(x,y,color='grey',alpha=0.3,s=0.8)
    parameter = np.polyfit(x, y, 1)
    f = np.poly1d(parameter)
    r, sig = scipy.stats.pearsonr(x, y)
    newx=np.linspace(np.min(x),np.max(x),20)
    ax.plot(newx, f(newx), ls='--', c='k')
    ax.text(0.10,0.80, "slope = %.2f\nR$^{2}$=%.2f$^{%s}$" %
             (parameter[0],r*r,getSigFlag(sig)),
             transform=ax.transAxes, c= 'k',fontsize=12)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    return ax


In [ ]:
fig, axes = plt.subplots(nrows=1,ncols=2,figsize=(8,3),sharex=False)
plot_settings()
fig.subplots_adjust(left = 0.08, right = 0.99,
                    bottom = 0.15, top = 0.99,
                    wspace= 0.2, hspace=0.05)
plot_scatter(axes[0],df['T31'],df['T21'],'$\Delta$LST_win31 [K]','$\Delta$LST_win21 [K]')
plot_scatter(axes[1],df['T31'],df['T41'],'$\Delta$LST_win31 [K]','$\Delta$LST_win41 [K]')

plt.savefig('../figures/Figure_S12.png',dpi=300)